# 首先是 Numpy 中的随机数

In [22]:
import numpy as np

In [ ]:
# 只要是种子一样的随机数生成器，生成的随机数序列都是一样的。

np.random.seed(0)

print(np.random.random())
print(np.random.random())
print(np.random.random())

np.random.seed(0)
print(np.random.random())
print(np.random.random())
print(np.random.random())

In [19]:
# 随机数生成器的状态是一个复杂的对象，包含了当前的状态信息。我们可以通过 `np.random.get_state()` 来获取当前的状态。
# 每次调用 `np.random.seed()` 都会重置随机数生成器的状态。

def print_truncated_random_state():
  """To avoid spamming the outputs, print only part of the state."""
  full_random_state = np.random.get_state()
  print(str(full_random_state)[:150], '...')

np.random.seed(42)
print_truncated_random_state()
np.random.seed(42)
print_truncated_random_state()

test = np.random.normal(0, 1, 10)
print(test.shape)
print_truncated_random_state()

('MT19937', array([        42, 3107752595, 1895908407, 3900362577, 3030691166,
       4081230161, 2732361568, 1361238961, 3961642104,  867618704,
     ...
('MT19937', array([        42, 3107752595, 1895908407, 3900362577, 3030691166,
       4081230161, 2732361568, 1361238961, 3961642104,  867618704,
     ...
(10,)
('MT19937', array([ 723970371, 1229153189, 4170412009, 2042542564, 3342822751,
       3177601514, 1210243767, 2648089330, 1412570585, 3849763494,
     ...


在上面的输出中，`MT19937` 是指 **Mersenne Twister 19937**，19937 对应于其周期长度。它是 NumPy 默认使用的伪随机数生成器（PRNG，Pseudo-Random Number Generator）的算法名称。

### 详细解释：
**Mersenne Twister** 是一种高效的伪随机数生成算法，具有以下特点：
   - 周期非常长（$2^{19937} - 1$）。
   - 生成的随机数分布均匀，质量较高。
   - 适合大多数科学计算和模拟任务。

In [20]:
# 可以设置生成随机数的形状

np.random.seed(0)
print(np.random.uniform(size=3))

[0.5488135  0.71518937 0.60276338]


In [23]:
# 一个个生成和一次性生成的效果是一样的

np.random.seed(0)
print("individually:", np.stack([np.random.uniform() for _ in range(3)]))

np.random.seed(0)
print("all at once: ", np.random.uniform(size=3))

individually: [0.5488135  0.71518937 0.60276338]
all at once:  [0.5488135  0.71518937 0.60276338]


# 下面是 JAX 的随机数生成器

In [ ]:
import jax
from jax import random

In [ ]:
# JAX 中，相同的 key 导致相同的随机数

key = random.key(42)
print(key)

print(random.normal(key))
print(random.normal(key))

In [38]:
# 一定要使用 `random.split()` 来分割 key

key = random.key(42) # 试试不加这一句并重复运行这个单元格？
for i in range(3):
  new_key, subkey = random.split(key)
  del key  # The old key is consumed by split() -- we must never use it again.

  val = random.normal(subkey)
  del subkey  # The subkey is consumed by normal().

  print(f"draw {i}: {val}")
  key = new_key  # new_key is safe to use in the next iteration.

draw 0: 0.6057640314102173
draw 1: -0.21089035272598267
draw 2: -0.3948981463909149


In [43]:
# 更多是这么书写代码：

print(f"Currently, key is :{key}")
key, subkey = random.split(key)
print(f"After split, key is: {key}")
print(f"After split, subkey is: {subkey}")

Currently, key is :Array((), dtype=key<fry>) overlaying:
[4277279094 3231914188]
After split, key is: Array((), dtype=key<fry>) overlaying:
[3895678382 3429534896]
After split, subkey is: Array((), dtype=key<fry>) overlaying:
[3515226245 1150219387]


In [55]:
# 或者 split 成多个 subkey

key = random.key(93)
key, *forty_two_subkeys = random.split(key, num=43)
# *符号 的作用是打包或解包不定长的参数或可迭代对象。
print(type(forty_two_subkeys), len(forty_two_subkeys))

key = random.key(93)
forty_three_subkeys = random.split(key, num=43)
print(type(forty_three_subkeys), len(forty_three_subkeys))

<class 'list'> 42
<class 'jax._src.prng.PRNGKeyArray'> 43


In [69]:
# JAX 中的随机数生成器不是顺序等价的

key = random.key(42)
subkeys = random.split(key, 3)
sequence1 = np.stack([random.normal(subkey) for subkey in subkeys])
print("individually:", sequence1)
sequence2 = jax.vmap(random.normal)(subkeys)
print("all at once: ", sequence2)

key = random.key(42)
print("all at once: ", random.normal(key, shape=(3,)))

individually: [0.07592554 0.60576403 0.4323065 ]
all at once:  [0.07592554 0.60576403 0.4323065 ]
all at once:  [-0.02830462  0.46713185  0.29570296]
